# Comparative Analysis: Agentic vs Baseline (kg-axel)

Notebook ini membandingkan hasil **Agentic Text2Cypher** dengan **Baseline** dari penelitian sebelumnya (kg-axel).

## Comparison:
- **Baseline**: kg-axel (single-pass inference, no self-correction)
- **Agentic**: kg-luthfi (iterative refinement with validation feedback)

## Key Questions:
1. Does the agentic approach improve Pass@1 rate?
2. How effective is self-correction (recovery rate)?
3. Which configurations benefit most from agentic approach?

In [ ]:
# Setup
import sys
import os
from pathlib import Path

project_root = Path.cwd().parent
os.chdir(project_root)

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [14, 6]
plt.rcParams['font.size'] = 10

print(f"Project root: {project_root}")

## 1. Load Baseline Results (kg-axel)

In [ ]:
# Baseline results from kg-axel research
# Source: /Users/tsimiscouse/Docs/Sarjana/Skripsi/kg-axel/text2cypher/combined_evaluation_metrics.xlsx

baseline_results = {
    "Zero-Shot_Full": {
        "pass_at_1_rate": 29.17,
        "kg_valid_rate": 80.77,
        "avg_bleu": 0.3636,
        "avg_rouge_l_f1": 0.5568,
        "avg_jaro_winkler": 0.7154,
        "avg_jaccard_output": 0.3958,
        "avg_llmetric_q": 40.74,
        "llmetric": 46.64,
    },
    "Zero-Shot_Nodes-Paths": {
        "pass_at_1_rate": 25.00,
        "kg_valid_rate": 71.15,
        "avg_bleu": 0.3229,
        "avg_rouge_l_f1": 0.5290,
        "avg_jaro_winkler": 0.6918,
        "avg_jaccard_output": 0.3077,
        "avg_llmetric_q": 36.33,
        "llmetric": 40.24,
    },
    "Zero-Shot_Only-Paths": {
        "pass_at_1_rate": 25.00,
        "kg_valid_rate": 65.38,
        "avg_bleu": 0.2942,
        "avg_rouge_l_f1": 0.5053,
        "avg_jaro_winkler": 0.6855,
        "avg_jaccard_output": 0.3269,
        "avg_llmetric_q": 35.68,
        "llmetric": 38.24,
    },
    "Few-Shot_Full": {
        "pass_at_1_rate": 48.94,
        "kg_valid_rate": 88.46,
        "avg_bleu": 0.4813,
        "avg_rouge_l_f1": 0.6440,
        "avg_jaro_winkler": 0.7754,
        "avg_jaccard_output": 0.5532,
        "avg_llmetric_q": 55.53,
        "llmetric": 63.49,
    },
    "Few-Shot_Nodes-Paths": {
        "pass_at_1_rate": 44.00,
        "kg_valid_rate": 88.46,
        "avg_bleu": 0.4592,
        "avg_rouge_l_f1": 0.6167,
        "avg_jaro_winkler": 0.7477,
        "avg_jaccard_output": 0.5000,
        "avg_llmetric_q": 52.81,
        "llmetric": 60.51,
    },
    "Few-Shot_Only-Paths": {
        "pass_at_1_rate": 45.83,
        "kg_valid_rate": 87.50,
        "avg_bleu": 0.4636,
        "avg_rouge_l_f1": 0.6212,
        "avg_jaro_winkler": 0.7580,
        "avg_jaccard_output": 0.5161,
        "avg_llmetric_q": 53.09,
        "llmetric": 60.75,
    },
    "CoT_Full": {
        "pass_at_1_rate": 44.00,
        "kg_valid_rate": 90.38,
        "avg_bleu": 0.4612,
        "avg_rouge_l_f1": 0.6174,
        "avg_jaro_winkler": 0.7546,
        "avg_jaccard_output": 0.4898,
        "avg_llmetric_q": 52.33,
        "llmetric": 60.51,
    },
    "CoT_Nodes-Paths": {
        "pass_at_1_rate": 42.31,
        "kg_valid_rate": 88.46,
        "avg_bleu": 0.4388,
        "avg_rouge_l_f1": 0.5993,
        "avg_jaro_winkler": 0.7397,
        "avg_jaccard_output": 0.4681,
        "avg_llmetric_q": 50.25,
        "llmetric": 57.98,
    },
    "CoT_Only-Paths": {
        "pass_at_1_rate": 44.23,
        "kg_valid_rate": 86.54,
        "avg_bleu": 0.4542,
        "avg_rouge_l_f1": 0.6084,
        "avg_jaro_winkler": 0.7463,
        "avg_jaccard_output": 0.4894,
        "avg_llmetric_q": 51.69,
        "llmetric": 59.02,
    },
}

df_baseline = pd.DataFrame([
    {"Configuration": k, **v} for k, v in baseline_results.items()
])

print("Baseline Results (kg-axel):")
display(df_baseline[['Configuration', 'pass_at_1_rate', 'kg_valid_rate', 'llmetric']])

## 2. Load Agentic Results

In [ ]:
# Load agentic results
results_dir = project_root / "results_v2"
metrics_path = results_dir / "metrics_summary.csv"

if metrics_path.exists():
    df_agentic = pd.read_csv(metrics_path)
    print(f"Loaded agentic results: {len(df_agentic)} configurations")
    display(df_agentic[['Configuration', 'Pass@1 (%)', 'KG Valid (%)', 'LLMetric']])
else:
    print("Agentic results not found.")
    print("Please run 02_evaluation_metrics.ipynb first.")
    df_agentic = None

## 3. Prepare Comparison Data

In [ ]:
# Map configuration names (handle potential naming differences)
config_mapping = {
    "Zero-Shot_Full": "Zero-Shot_Full",
    "Zero-Shot_Nodes-Paths": "Zero-Shot_Nodes-Paths",
    "Zero-Shot_Only-Paths": "Zero-Shot_Only-Paths",
    "Few-Shot_Full": "Few-Shot_Full",
    "Few-Shot_Nodes-Paths": "Few-Shot_Nodes-Paths",
    "Few-Shot_Only-Paths": "Few-Shot_Only-Paths",
    "CoT_Full": "CoT_Full",
    "CoT_Nodes-Paths": "CoT_Nodes-Paths",
    "CoT_Only-Paths": "CoT_Only-Paths",
}

if df_agentic is not None:
    # Normalize config names
    df_agentic['Config_Normalized'] = df_agentic['Configuration'].map(
        lambda x: config_mapping.get(x, x)
    )
    
    # Create comparison dataframe
    comparison_data = []
    
    for config_name in baseline_results.keys():
        baseline = baseline_results[config_name]
        
        # Find matching agentic result
        agentic_row = df_agentic[df_agentic['Config_Normalized'] == config_name]
        
        if len(agentic_row) > 0:
            agentic = agentic_row.iloc[0]
            
            comparison_data.append({
                "Configuration": config_name,
                "Baseline Pass@1": baseline['pass_at_1_rate'],
                "Agentic Pass@1": agentic['Pass@1 (%)'],
                "Pass@1 Delta": agentic['Pass@1 (%)'] - baseline['pass_at_1_rate'],
                "Baseline KG Valid": baseline['kg_valid_rate'],
                "Agentic KG Valid": agentic['KG Valid (%)'],
                "KG Valid Delta": agentic['KG Valid (%)'] - baseline['kg_valid_rate'],
                "Baseline LLMetric": baseline['llmetric'],
                "Agentic LLMetric": agentic['LLMetric'],
                "LLMetric Delta": agentic['LLMetric'] - baseline['llmetric'],
                "Avg Iterations": agentic['Avg Iterations'],
                "Recovery Rate": agentic.get('Recovery Rate (%)', 0),
            })
        else:
            comparison_data.append({
                "Configuration": config_name,
                "Baseline Pass@1": baseline['pass_at_1_rate'],
                "Agentic Pass@1": None,
                "Pass@1 Delta": None,
                "Baseline KG Valid": baseline['kg_valid_rate'],
                "Agentic KG Valid": None,
                "KG Valid Delta": None,
                "Baseline LLMetric": baseline['llmetric'],
                "Agentic LLMetric": None,
                "LLMetric Delta": None,
                "Avg Iterations": None,
                "Recovery Rate": None,
            })
    
    df_comparison = pd.DataFrame(comparison_data)
    print("\nComparison: Baseline vs Agentic")
    display(df_comparison)

## 4. Visualizations

In [ ]:
# Pass@1 Comparison: Baseline vs Agentic
if df_agentic is not None and len(df_comparison) > 0:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    x = np.arange(len(df_comparison))
    width = 0.35
    
    baseline_vals = df_comparison['Baseline Pass@1'].values
    agentic_vals = df_comparison['Agentic Pass@1'].fillna(0).values
    
    bars1 = ax.bar(x - width/2, baseline_vals, width, label='Baseline (kg-axel)', color='steelblue')
    bars2 = ax.bar(x + width/2, agentic_vals, width, label='Agentic (kg-luthfi)', color='coral')
    
    ax.set_xlabel('Configuration', fontsize=12)
    ax.set_ylabel('Pass@1 Rate (%)', fontsize=12)
    ax.set_title('Pass@1 Rate: Baseline vs Agentic', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(df_comparison['Configuration'], rotation=45, ha='right')
    ax.legend()
    
    # Add value labels
    for bar, val in zip(bars1, baseline_vals):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{val:.1f}', ha='center', va='bottom', fontsize=8)
    for bar, val in zip(bars2, agentic_vals):
        if val > 0:
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                    f'{val:.1f}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.savefig(results_dir / 'comparison_pass_at_1.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Improvement Delta
if df_agentic is not None and len(df_comparison) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Pass@1 Delta
    deltas = df_comparison['Pass@1 Delta'].fillna(0).values
    colors = ['green' if d > 0 else 'red' for d in deltas]
    axes[0].bar(df_comparison['Configuration'], deltas, color=colors)
    axes[0].set_xlabel('Configuration')
    axes[0].set_ylabel('Delta (%)')
    axes[0].set_title('Pass@1 Improvement (Agentic - Baseline)')
    axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[0].set_xticklabels(df_comparison['Configuration'], rotation=45, ha='right')
    
    # KG Valid Delta
    deltas = df_comparison['KG Valid Delta'].fillna(0).values
    colors = ['green' if d > 0 else 'red' for d in deltas]
    axes[1].bar(df_comparison['Configuration'], deltas, color=colors)
    axes[1].set_xlabel('Configuration')
    axes[1].set_ylabel('Delta (%)')
    axes[1].set_title('KG Valid Improvement (Agentic - Baseline)')
    axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[1].set_xticklabels(df_comparison['Configuration'], rotation=45, ha='right')
    
    # LLMetric Delta
    deltas = df_comparison['LLMetric Delta'].fillna(0).values
    colors = ['green' if d > 0 else 'red' for d in deltas]
    axes[2].bar(df_comparison['Configuration'], deltas, color=colors)
    axes[2].set_xlabel('Configuration')
    axes[2].set_ylabel('Delta')
    axes[2].set_title('LLMetric Improvement (Agentic - Baseline)')
    axes[2].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[2].set_xticklabels(df_comparison['Configuration'], rotation=45, ha='right')
    
    plt.tight_layout()
    plt.savefig(results_dir / 'comparison_improvement_delta.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Self-Correction Impact
if df_agentic is not None and len(df_comparison) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Recovery Rate vs Improvement
    valid_data = df_comparison.dropna(subset=['Recovery Rate', 'Pass@1 Delta'])
    
    if len(valid_data) > 0:
        axes[0].scatter(valid_data['Recovery Rate'], valid_data['Pass@1 Delta'], 
                       s=100, c='coral', alpha=0.7)
        
        for _, row in valid_data.iterrows():
            axes[0].annotate(row['Configuration'].split('_')[0], 
                           (row['Recovery Rate'], row['Pass@1 Delta']),
                           fontsize=8)
        
        axes[0].set_xlabel('Recovery Rate (%)')
        axes[0].set_ylabel('Pass@1 Improvement (%)')
        axes[0].set_title('Recovery Rate vs Pass@1 Improvement')
        axes[0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    
    # Iterations vs Recovery Rate
    valid_data = df_comparison.dropna(subset=['Avg Iterations', 'Recovery Rate'])
    
    if len(valid_data) > 0:
        axes[1].scatter(valid_data['Avg Iterations'], valid_data['Recovery Rate'],
                       s=100, c='mediumpurple', alpha=0.7)
        
        for _, row in valid_data.iterrows():
            axes[1].annotate(row['Configuration'].split('_')[0],
                           (row['Avg Iterations'], row['Recovery Rate']),
                           fontsize=8)
        
        axes[1].set_xlabel('Average Iterations')
        axes[1].set_ylabel('Recovery Rate (%)')
        axes[1].set_title('Iterations vs Recovery Rate')
    
    plt.tight_layout()
    plt.savefig(results_dir / 'comparison_self_correction.png', dpi=150, bbox_inches='tight')
    plt.show()

## 5. Statistical Summary

In [ ]:
# Overall Statistics
if df_agentic is not None and len(df_comparison) > 0:
    valid_comparison = df_comparison.dropna()
    
    print("="*60)
    print("OVERALL STATISTICS")
    print("="*60)
    
    print(f"\nConfigurations compared: {len(valid_comparison)}")
    
    # Average improvements
    avg_pass1_delta = valid_comparison['Pass@1 Delta'].mean()
    avg_kg_valid_delta = valid_comparison['KG Valid Delta'].mean()
    avg_llmetric_delta = valid_comparison['LLMetric Delta'].mean()
    
    print(f"\nAverage Pass@1 Improvement: {avg_pass1_delta:+.2f}%")
    print(f"Average KG Valid Improvement: {avg_kg_valid_delta:+.2f}%")
    print(f"Average LLMetric Improvement: {avg_llmetric_delta:+.2f}")
    
    # Improvements count
    improved_pass1 = (valid_comparison['Pass@1 Delta'] > 0).sum()
    improved_llmetric = (valid_comparison['LLMetric Delta'] > 0).sum()
    
    print(f"\nConfigurations with improved Pass@1: {improved_pass1}/{len(valid_comparison)}")
    print(f"Configurations with improved LLMetric: {improved_llmetric}/{len(valid_comparison)}")
    
    # Best and worst improvements
    best_improvement = valid_comparison.loc[valid_comparison['Pass@1 Delta'].idxmax()]
    worst_improvement = valid_comparison.loc[valid_comparison['Pass@1 Delta'].idxmin()]
    
    print(f"\nBest Pass@1 improvement: {best_improvement['Configuration']} ({best_improvement['Pass@1 Delta']:+.2f}%)")
    print(f"Worst Pass@1 change: {worst_improvement['Configuration']} ({worst_improvement['Pass@1 Delta']:+.2f}%)")
    
    # Recovery effectiveness
    avg_recovery = valid_comparison['Recovery Rate'].mean()
    avg_iterations = valid_comparison['Avg Iterations'].mean()
    
    print(f"\nAverage Recovery Rate: {avg_recovery:.2f}%")
    print(f"Average Iterations: {avg_iterations:.2f}")

In [ ]:
# Analysis by Prompt Type
if df_agentic is not None and len(df_comparison) > 0:
    valid_comparison = df_comparison.dropna()
    valid_comparison['Prompt'] = valid_comparison['Configuration'].apply(lambda x: x.split('_')[0])
    
    prompt_stats = valid_comparison.groupby('Prompt').agg({
        'Baseline Pass@1': 'mean',
        'Agentic Pass@1': 'mean',
        'Pass@1 Delta': 'mean',
        'LLMetric Delta': 'mean',
        'Recovery Rate': 'mean',
    }).round(2)
    
    print("\n" + "="*60)
    print("ANALYSIS BY PROMPT TYPE")
    print("="*60)
    display(prompt_stats)

## 6. Conclusions

In [ ]:
# Generate conclusions
if df_agentic is not None and len(df_comparison) > 0:
    valid_comparison = df_comparison.dropna()
    
    print("="*60)
    print("CONCLUSIONS")
    print("="*60)
    
    avg_pass1_delta = valid_comparison['Pass@1 Delta'].mean()
    avg_recovery = valid_comparison['Recovery Rate'].mean()
    improved_count = (valid_comparison['Pass@1 Delta'] > 0).sum()
    total_count = len(valid_comparison)
    
    print(f"\n1. EFFECTIVENESS OF AGENTIC APPROACH:")
    if avg_pass1_delta > 0:
        print(f"   - Agentic approach IMPROVES Pass@1 rate by average of {avg_pass1_delta:.2f}%")
    else:
        print(f"   - Agentic approach shows mixed results (avg delta: {avg_pass1_delta:.2f}%)")
    
    print(f"   - {improved_count}/{total_count} configurations show improvement")
    
    print(f"\n2. SELF-CORRECTION CAPABILITY:")
    print(f"   - Average Recovery Rate: {avg_recovery:.2f}%")
    print(f"   - Self-correction {'is effective' if avg_recovery > 30 else 'has limited effectiveness'}")
    
    # Best performing prompt type
    valid_comparison['Prompt'] = valid_comparison['Configuration'].apply(lambda x: x.split('_')[0])
    best_prompt = valid_comparison.groupby('Prompt')['Pass@1 Delta'].mean().idxmax()
    best_prompt_delta = valid_comparison.groupby('Prompt')['Pass@1 Delta'].mean().max()
    
    print(f"\n3. BEST PERFORMING PROMPT TYPE:")
    print(f"   - {best_prompt} shows best improvement ({best_prompt_delta:+.2f}%)")
    
    print(f"\n4. RECOMMENDATION:")
    if avg_pass1_delta > 5:
        print("   - Agentic approach is RECOMMENDED for Text2Cypher tasks")
    elif avg_pass1_delta > 0:
        print("   - Agentic approach provides MODEST improvements")
    else:
        print("   - Consider optimizing self-correction strategy")

## 7. Export Results

In [ ]:
# Export comparison results
if df_agentic is not None and len(df_comparison) > 0:
    # Save comparison CSV
    df_comparison.to_csv(results_dir / 'comparative_analysis.csv', index=False)
    print(f"Comparison saved to: {results_dir / 'comparative_analysis.csv'}")
    
    # Save to Excel
    try:
        with pd.ExcelWriter(results_dir / 'comparative_analysis.xlsx', engine='openpyxl') as writer:
            df_comparison.to_excel(writer, sheet_name='Comparison', index=False)
            df_baseline.to_excel(writer, sheet_name='Baseline', index=False)
            if 'df_metrics' in dir():
                df_agentic.to_excel(writer, sheet_name='Agentic', index=False)
        print(f"Excel saved to: {results_dir / 'comparative_analysis.xlsx'}")
    except Exception as e:
        print(f"Could not save Excel: {e}")

In [ ]:
print("\n" + "="*60)
print("COMPARATIVE ANALYSIS COMPLETE")
print("="*60)
print(f"\nResults saved to: {results_dir}")
print("\nFiles generated:")
print("  - comparative_analysis.csv")
print("  - comparative_analysis.xlsx")
print("  - comparison_*.png (visualizations)")